# Caso de estudio 1. Optimización de una caja.

### **Librearías**

In [11]:
import numpy as np
import pandas as pd

## **Generar poblacion Incial**

In [276]:
def generar_poblacion(n, l=9):
    """Crea una población inicial de crmosomas binrarios.

    Args:
        n (int): número de individuos en la población,
        l (int): Longitud de cada cromosoma. Default l = 9

    Returns:
        np.array(): Matriz n x l
    """
    return np.random.randint(2, size=(n, l))

## **Decodificación de Cromosomas**

Cada cromosoma binario es decodificado en valores decimales que representan las dimensiones `l` (longitud), `w` (ancho), y `h` (altura) de la caja. La función `bin2dec` convierte un segmento binario en un valor decimal dentro de un rango definido, y `decodificar_cromosoma` aplica esta conversión a las tres dimensiones.

Fórmula de decodificación:

$$
x = L + \frac{\text{valor decimal}}{2^{n}-1} * (U - L)
$$

Donde:
$U$ es el valor superior del rango.

$L$ el valor inferior del rango.

$n$ el numero de bits (genes).

In [283]:
def bin2dec(cadena, _min, _max):
    """Convierte un segmento binario en un valor decimal dentro de un rango definido.

    Args:
        cadena (np.array): Cromosoma de n poblaciones.
        _min (int): Valor mínimo del rango.
        _max (int): Valor máximo del rango.

    Returns:
        _type_: _description_
    """
    longitud = len(cadena)
    valor_posicional = np.asarray([2**i for i in range(longitud)])[::-1]
    valor_decimal = cadena.dot(valor_posicional)
    max_binario = (2 ** longitud) - 1
    return _min + (valor_decimal / max_binario) * (_max - _min)

def decodificar_cromosoma(cromosoma, _min, _max):
    """Aplica la conversión don2dec a las tres dimensiones.

    Args:
        cromosoma (_type_): _description_
        _min (int): Valor mínimo del rango.
        _max (int): Valor máximo del rango.

    Returns:
        np.array: np.array([])
    """
    longitud = len(cromosoma) // 3
    l = bin2dec(cromosoma[:longitud], _min, _max)
    w = bin2dec(cromosoma[longitud:2*longitud], _min, _max)
    h = bin2dec(cromosoma[2*longitud:], _min, _max)
    return np.array([l, w, h])

## **Función de Aptitud**
La aptitud de un cromosoma se calcula como el volumen de la caja, pero con una penalización si el área superficial excede un límite (20 en este caso).

In [452]:
def funcion_aptitud(l, w, h):
    """Función de aptitud

    Args:
        l, w y h se obtienen de la función decodificador_cromosoma

    Returns:
        _type_: _description_
    """
    volumen = l * w * h
    area = 2 * (l * w + l * h + w * h)
    if area < 20:
        return 0  # Penalización si no se cumple la restricción
    return volumen

## **Selección de padres por Ruleta**

In [440]:
# Selección de padres
def seleccion(poblacion, aptitudes):
    total_aptitud = np.sum(aptitudes)

    # Evitar división por cero si todas las aptitudes son cero
    if total_aptitud == 0:
        probabilidades = np.ones(len(poblacion)) / len(poblacion)
    else:
        probabilidades = aptitudes / total_aptitud

    # Selecciona dos individuos con probabilidad proporcional a su aptitud
    indices = np.random.choice(len(poblacion), size=2, p=probabilidades)
    return [poblacion[i] for i in indices] # Return a list of selected individuals

## **Cruce de puntos**

In [544]:
def cruce(padre1, padre2,longitud_cromosomas,cruce_prob):
    """Fusiona información de dos genotipos parentales en uno o dos genotipos descendientes dependiendo de su probaliidad.

    Args:
        padre1 (float): Genotipo del padre 1.
        padre2 (float): Genotipo del padre 2.

    Returns:
        tuple: (hijo1, hijo2)
    """
    if np.random.rand() < cruce_prob:
        punto_cruce = np.random.randint(1, longitud_cromosoma)
        hijo1 = np.concatenate((padre1[:punto_cruce], padre2[punto_cruce:]))
        hijo2 = np.concatenate((padre2[:punto_cruce], padre1[punto_cruce:]))
        return hijo1, hijo2
    else:
        return padre1, padre2

## **Mutación**

In [563]:
def mutacion_un_bit(individuo):
    """Se aplica a un genotipo y entraga un mutante.

    Args:
        individuo (numpy): Cromosoma del individuo a cambiar,

    Returns:
        numpy.array: numpy.array(mutacion_individuo)
    """
    bit_a_mutar = np.random.randint(len(individuo))
    individuo[bit_a_mutar] = 1 - individuo[bit_a_mutar]
    return individuo

## Reemplazo Elitista

In [564]:
def reemplazo_elitista(poblacion, nueva_poblacion, num_elites):
    """El reemplazo elitista asegura que los 
       mejores individuos de la población actual se preserven para la siguiente generación.

    Args:
        poblacion (np.array(n*m)): Población
        poblacion (np.array(n*m)): Nueva población
        num_elites (int): tamaño del grupo élite.

    Returns:
        _type_: _description_
    """
    poblacion_ordenada = sorted(poblacion, key=lambda ind: funcion_aptitud(*decodificar_cromosoma(ind, 0, 5)), reverse=True)
    nueva_poblacion_ordenada = sorted(nueva_poblacion, key=lambda ind: funcion_aptitud(*decodificar_cromosoma(ind, 0, 5)), reverse=True)
    elitistas = poblacion_ordenada[:num_elites]
    nueva_poblacion_reemplazada = elitistas + nueva_poblacion_ordenada[:len(poblacion) - num_elites]
    return np.array(nueva_poblacion_reemplazada)

## **Algoritmo Genético**

In [588]:
def algoritmo_genetico(poblacion_size = 20, longitud_cromosomas = 9, 
                       generaciones = 100, num_elites = 10, cruce_prob=.07):
    
    poblacion = generar_poblacion(poblacion_size,l=longitud_cromosomas) # TODO Crear una población.

    
    for generacion in range(generaciones): # TODO Iteración de n generaciones
        
        nueva_poblacion = []
        
        aptitudes = [np.array(funcion_aptitud(*decodificar_cromosoma(cromosoma,0,5))) for cromosoma in poblacion]
        
        #print(f"Generacion: {generacion+1}")
        for _ in range(poblacion_size // 2):
            padre1, padre2 = seleccion(poblacion,aptitudes)
            hijo1, hijo2 = cruce(padre1,padre2,
                                 longitud_cromosomas=longitud_cromosomas,
                                 cruce_prob=cruce_prob)
            nueva_poblacion.append(mutacion_un_bit(hijo1))
            nueva_poblacion.append(mutacion_un_bit(hijo2))
        
        mejor_individuo = poblacion[np.argmax([funcion_aptitud(*decodificar_cromosoma(cromosoma,0,5)) for cromosoma in poblacion])]
        mejor_aptitud = funcion_aptitud(*decodificar_cromosoma(mejor_individuo,0,5))
        
        print(f'Generación {generacion + 1}: Mejor aptitud = {mejor_aptitud} Mejor individuo = {mejor_individuo}')
        
        # Reemplazo Elitista.
        poblacion = reemplazo_elitista(poblacion, nueva_poblacion, num_elites)  
        
    return mejor_individuo

In [594]:
mejor_individuo = algoritmo_genetico(poblacion_size=100,longitud_cromosomas=24,generaciones=17,num_elites=5,cruce_prob=0.7)

Generación 1: Mejor aptitud = 92.40035883634499 Mejor individuo = [1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 0]
Generación 2: Mejor aptitud = 95.15570934256054 Mejor individuo = [1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0]
Generación 3: Mejor aptitud = 106.75381263616559 Mejor individuo = [1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 0]
Generación 4: Mejor aptitud = 113.93053953607587 Mejor individuo = [1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0]
Generación 5: Mejor aptitud = 113.93053953607587 Mejor individuo = [1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0]
Generación 6: Mejor aptitud = 116.81231200669424 Mejor individuo = [1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0]
Generación 7: Mejor aptitud = 115.7534281686531 Mejor individuo = [1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0]
Generación 8: Mejor aptitud = 120.63233597937445 Mejor individuo = [1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0]
Generación 9: Mejor aptitud = 120.63233597937445 Mejor individuo = 

In [598]:
print(f"La mejor solucion para optimizar la caja es: {decodificar_cromosoma(mejor_individuo,0,5)}")

La mejor solucion para optimizar la caja es: [4.96078431 4.98039216 4.92156863]
